<a href="https://colab.research.google.com/github/hhhwmws0117/Chat-personal-embodiment/blob/main/2.0/double_chatbot_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chatharuhi datasets
!pip -q install transformers openai tiktoken langchain chromadb

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "sk-NYJ30H6jdMFO32c0a"

In [ ]:
!mkdir -p ./character/user1/story_txts
%cd ./character/user1/story_txts
!curl -s "https://api.github.com/repos/LC1332/Chat-Haruhi-Suzumiya/contents/characters/ayaka/texts?ref=main" | grep -oE 'https://github.com[^"]+.txt' | xargs -n 1 curl -O -L

In [2]:
%cd ..
!wget -q https://github.com/LC1332/Chat-Haruhi-Suzumiya/raw/main/characters/ayaka/system_prompt.txt

/content/character/user1


In [15]:
from chatharuhi import ChatHaruhi

story_text_folder = '/content/character/user1/story_txts'
system_prompt = '/content/character/user1/system_prompt.txt'

chatbot = ChatHaruhi( system_prompt = system_prompt,\
                      llm = 'openai' ,\
                      story_text_folder = story_text_folder,\
                      verbose = True)


starting extract embedding... for 117 files


RateLimitError: ignored

以下的函数不负责储存数据，只负责进行chat

在外层需要一个文件夹用以储存

子路径story_txts下的灵魂测试 + dialogue example，以txt形式储存

子路径system_prompt.txt



In [ ]:
def double_chatbot_chat(role_name_1, role_name_2, role_1_story_txt = None, role_2_story_txt = None, chat_round = 10, llm = "openai" ):
  NAME_DICT = {'汤师爷': 'tangshiye', '慕容复': 'murongfu', '李云龙': 'liyunlong', 'Luna': 'Luna', '王多鱼': 'wangduoyu',
            'Ron': 'Ron', '鸠摩智': 'jiumozhi', 'Snape': 'Snape',
            '凉宫春日': 'haruhi', 'Malfoy': 'Malfoy', '虚竹': 'xuzhu', '萧峰': 'xiaofeng', '段誉': 'duanyu',
            'Hermione': 'Hermione', 'Dumbledore': 'Dumbledore', '王语嫣': 'wangyuyan',
            'Harry': 'Harry', 'McGonagall': 'McGonagall', '白展堂': 'baizhantang', '佟湘玉': 'tongxiangyu',
            '郭芙蓉': 'guofurong', '旅行者': 'wanderer', '钟离': 'zhongli',
            '胡桃': 'hutao', 'Sheldon': 'Sheldon', 'Raj': 'Raj', 'Penny': 'Penny', '韦小宝': 'weixiaobao',
            '乔峰': 'qiaofeng', '神里绫华': 'ayaka', '雷电将军': 'raidenShogun', '于谦': 'yuqian'}
  if (role_1_story_txt is None) and (role_name_1 not in list(NAME_DICT.keys())):
    raise FileNotFoundError(f"没有找到{role_name_1}")
  elif (role_1_story_txt is None) and (role_name_1 in list(NAME_DICT.keys())):
    chatbot_1 = ChatHaruhi( role_name = NAME_DICT[role_name_1], \
                      llm = llm, verbose = True)
  else:
    file_path_system_prompt = role_1_story_txt + "/system_prompt.txt"
    with open(file_path_system_prompt, "r") as file:
        system_prompt = file.read()
    chatbot_1 = ChatHaruhi( system_prompt = system_prompt,\
                      llm = llm ,\
                      story_db = role_1_story_txt + "/story_txts",\
                      verbose = True)

  if (role_2_story_txt is None) and (role_name_2 not in list(NAME_DICT.keys())):
    raise FileNotFoundError(f"没有找到{role_name_2}")
  elif (role_2_story_txt is None) and (role_name_2 in list(NAME_DICT.keys())):
    chatbot_2 = ChatHaruhi( role_name = NAME_DICT[role_name_2], \
                      llm = llm, verbose = True)
  else:
    file_path_system_prompt = role_2_story_txt + "/system_prompt.txt"
    with open(file_path_system_prompt, "r") as file:
        system_prompt = file.read()
    chatbot_2 = ChatHaruhi( system_prompt = system_prompt,\
                      llm = llm ,\
                      story_db = role_2_story_txt + "/story_txts",\
                      verbose = True)
  response_1 = chatbot_1.chat(role=role_name_2,text ='你好！我是' + role_name_2 + "！ 很高兴认识你！")
  response_2 = chatbot_2.chat(role=role_name_1,text =response_1)
  for i in range(chat_round-1):
    response_1 = chatbot_1.chat(role=role_name_2,text =response_2)
    response_2 = chatbot_2.chat(role=role_name_1,text =response_1)



  return chatbot_2.dialogue_history